In [14]:
! add-apt-repository -y ppa:savoury1/ffmpeg4
! apt-get -qq install -y ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,259 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:14 http://ppa.launc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#The output of this section is the CSV files with the data to be handle by the model
# As this is the most time consuming task, only enable it if you don't have the CSV files yet
CREATE_CSV_FILES = True

In [ ]:
# Defines the names of the CSV files
TRAIN_CSV_FILE = "train.csv"
TEST_CSV_FILE = "test.csv"
MORE_TRAIN_CSV_FILE = "more_train.csv"
MORE_TEST_CSV_FILE = "more_test.csv"

In [10]:
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
from matplotlib import cm
import pandas as pd
import librosa
import zipfile
import csv
import os

In [12]:
with zipfile.ZipFile('/content/ravasconcelos spoken-digits-recognition master data-recordings.zip', 'r') as zip_ref:
    zip_ref.extractall()
    print("process completed")

process completed


AttributeError: ignored

In [ ]:
def extractWavFeatures(soundFilesFolder, csvFileName):
  # print("The features of the files in the folder "+soundFilesFolder+" will be saved to "+csvFileName)
  header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
  #mfccs represent the envelope of short time power spectrum which manifists from the shape of vocal tract 
  for i in range(1, 21):
          header += f' mfcc{i}'
  #target variable
  header += ' label'
  # print('CSV Header: ', header)
  #split the string into a list of words
  header = header.split()
  # print('CSV Header: ', header)
  file = open(csvFileName, 'w', newline='')
  writer = csv.writer(file)
  writer.writerow(header)
  genres = '1 2 3 4 5 6 7 8 9 0'.split()
  #listdir() returns a list containing the names of the entries(files) in the directory given by path.
  i=0
  for filename in os.listdir(soundFilesFolder):
   
    sound_file = os.path.join(soundFilesFolder, filename)
    #mono set to true which converts the  signal to monoe
    #duration:only load up to this much audio (in seconds)
    sr, y = scipy.io.wavfile.read(sound_file, mono=True, duration=30)
    print(i)
    i+=1
  #   # #remove leading and trailing silence
  #   y, index = librosa.effects.trim(y)
  #   chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
  #   rmse = librosa.feature.rms(y=y)
  #   spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
  #   spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
  #   rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
  #   zcr = librosa.feature.zero_crossing_rate(y)
  #   mfcc = librosa.feature.mfcc(y=y, sr=sr)
  #   to_append = f'{filename} {np.mean(chroma_stft)}{np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
  #   for e in mfcc:
  #       to_append += f'{np.mean(e)}'
  # writer.writerow(to_append.split())
  # file.close()

In [13]:
if (CREATE_CSV_FILES == True):
    extractWavFeatures("/content/train", TRAIN_CSV_FILE)
    extractWavFeatures("/content/test", TEST_CSV_FILE)
    extractWavFeatures("/content/moreSpeakersTrain", MORE_TRAIN_CSV_FILE)
    extractWavFeatures("/content/moreSpeakersTest", MORE_TEST_CSV_FILE)
    print("CSV files are created")
else:
    print("CSV files creation is skipped")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


NoBackendError: ignored